In [1]:
from Mollification import *

import trimesh
import os
import time

import meshio as mio

from buildGlueMap import *

root_folder = os.getcwd()

In [11]:
G = np.array([[[-1, -1], [-1, -1], [1, 0]], [[0,2], [2, 0], [3, 0]], [[1, 1], [-1, -1], [-1, -1]], [[1, 2], [-1, -1], [-1, -1]]])
FL = np.zeros((4, 3))
#E2FL = np.array([(0, 0), (0, 1), (0, 2), (1, 1), (2, 2), (2, 1), (1, 2), (3, 1), (3, 2)])
E2FL = np.array([[0, 0], [0, 1], [0, 2], [1, 1], [2, 2], [2, 1], [1, 2]])
#LE = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8])
LE = np.array([0, 1, 2, 3, 4, 5, 6])

FL2E = np.array([[[0], [1], [2]], [[2], [3], [6]], [[3], [5], [4]], [[6], [7], [8]]])



In [12]:
FL[E2FL[:,0], E2FL[:,1]] = LE
print(FL)

[[0. 1. 2.]
 [0. 3. 6.]
 [0. 5. 4.]
 [0. 0. 0.]]


In [4]:
((G[E2FL[:,0], E2FL[:,1]][:,0] != -1) & (G[E2FL[:,0], E2FL[:,1]][:,1] != -1))

array([False, False,  True,  True, False, False,  True])

In [2]:
FS = (E2FL[:,0], E2FL[:,1])
print(FS)

NameError: name 'E2FL' is not defined

In [6]:
FSP = G[FS][((G[FS][:,0] != -1) & (G[FS][:,1] != -1)), :]
FSP = (FSP[:, 0], FSP[:, 1])
print(FSP)

(array([1, 2, 3]), array([0, 0, 0]))


In [7]:
FL[FSP] = LE[FL2E[FSP]][:,0]
print(FL)

[[0. 1. 2.]
 [2. 3. 6.]
 [3. 5. 4.]
 [6. 0. 0.]]


In [4]:
mesh = mio.read("../data/goathead.obj")
F = mesh.cells_dict["triangle"]
V = mesh.points

In [5]:
print(igl.euler_characteristic(F))

1


In [5]:
print(FL)

[[141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [200.         141.42135624 141.42135624]
 [141.42135624 141.42135624 200.        ]
 [200.         141.42135624 141.42135624]
 [141.42135624 141.42135624 200.        ]
 [200.         141.42135624 141.42135624]
 [141.42135624 141.42135624 200.        ]
 [200.         141.42135624 141.42135624]
 [141.42135624 141.42135624 200.        ]
 [200.         141.42135624 141.42135624]
 [141.42135624 141.42135624 200.        ]
 [200.         141.42135624 141.42135624]
 [141.42135624 141.42135624 200.  

In [5]:
print(FLG)

[[141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [200.         141.42135624 141.42135624]
 [141.42135624 141.42135624 200.        ]
 [200.         141.42135624 141.42135624]
 [141.42135624 141.42135624 200.        ]
 [200.         141.42135624 141.42135624]
 [141.42135624 141.42135624 200.        ]
 [200.         141.42135624 141.42135624]
 [141.42135624 141.42135624 200.        ]
 [200.         141.42135624 141.42135624]
 [141.42135624 141.42135624 200.        ]
 [200.         141.42135624 141.42135624]
 [141.42135624 141.42135624 200.  

In [3]:
delta = 1e-2
FL = igl.edge_lengths(V, F)

In [4]:
G = build_gluing_map(F)

end5 = time.time()

FLG, nMollG, nIterG = IntrinsicMollificationFL(FL, G, delta, scheme=MOLLIFICATION_SCHEME.GLOBAL_OPTIMIZATION_MANHATTAN)
end6 = time.time()
print("Global optimization with Manhattan distance took: ", end6 - end5, " seconds.")

FLG2, nMollG2, nIterG2 = IntrinsicMollificationFL(FL, G, delta, scheme=MOLLIFICATION_SCHEME.GLOBAL_OPTIMIZATION_EUCLIDEAN)
end7 = time.time()
print("Global optimization with Manhattan distance took: ", end7 - end6, " seconds.")


FLL, nMollL, _ = IntrinsicMollificationFL(FL, None, delta, scheme=MOLLIFICATION_SCHEME.LOCAL_SCHEMES, local_scheme=MOLLIFICATION_LOCAL_SCHEME.ONE_BY_ONE_INTERPOLATED)

end8 = time.time()
print("Local optimization with one-by-one interpolation took: ", end8 - end7, " seconds.")

FLC, nMollC, _ = IntrinsicMollificationFL(FL, None, delta, scheme=MOLLIFICATION_SCHEME.CONSTANT_EPSILON)
end9 = time.time()
print("Constant epsilon took: ", end9 - end8, " seconds.")


-81.23323676637028 0.016094757082487303
Global optimization with Manhattan distance took:  0.004501819610595703  seconds.
180 [ 1. -1.  1. -1. -1.  1. -1.  1. -1. -1.  1. -1.  1. -1. -1.  1. -1.  1.
 -1. -1.  1. -1.  1. -1. -1.  1. -1.  1. -1. -1.  1. -1.  1. -1. -1.  1.
 -1.  1. -1. -1.  1. -1.  1. -1. -1.  1. -1.  1. -1. -1.  1. -1.  1. -1.
 -1.  1. -1.  1. -1. -1.  1. -1.  1. -1. -1.  1. -1.  1. -1. -1.  1. -1.
  1. -1. -1.  1. -1.  1. -1. -1.  1. -1.  1. -1. -1.  1. -1.  1. -1. -1.
  1. -1.  1. -1. -1.  1. -1.  1. -1. -1.  1. -1.  1. -1. -1.  1. -1.  1.
 -1. -1.  1. -1.  1. -1. -1.  1. -1.  1. -1. -1.  1. -1.  1. -1. -1.  1.
 -1.  1. -1. -1.  1. -1.  1. -1. -1.  1. -1.  1. -1. -1.  1. -1.  1. -1.
 -1.  1. -1.  1. -1. -1.  1. -1.  1. -1. -1.  1. -1.  1. -1. -1.  1. -1.
  1. -1. -1.  1. -1.  1. -1. -1.  1. -1.  1. -1. -1.  1. -1.  1. -1. -1.]
180 [  0   0   1   1   2   3   3   4   4   5   6   6   7   7   8   9   9  10
  10  11  12  12  13  13  14  15  15  16  16  17  18  18  19  19  

AttributeError: 'dict' object has no attribute 'x'

In [6]:
np.linalg.norm(FLG - FL) / np.linalg.norm(FL)

0.0

In [7]:
np.linalg.norm(FLL - FL)/ np.linalg.norm(FL)

0.0

In [8]:
np.linalg.norm(FLC - FL) / np.linalg.norm(FL)

0.0

In [7]:
print(cvx.spmatrix([-1.0, 2, 4], [0, 1, 2], [0, 0, 1]))

[-1.00e+00     0         0    ]
[ 2.00e+00     0         0    ]
[    0         0      4.00e+00]



In [11]:
a = [1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1] 
b = [0, 0, 0, 1, 1, 1, 2, 3, 3, 3, 4, 4, 4, 5, 6, 6, 6, 7, 7, 7, 8, 9, 9, 9, 10, 10, 10, 11, 12, 12, 12, 13, 13, 13, 14, 15, 15, 15, 16, 16, 16, 17, 18, 18, 18, 19, 19, 19, 20, 21, 21, 21, 22, 22, 22, 23, 24, 24, 24, 25, 25, 25, 26, 27, 27, 27, 28, 28, 28, 29, 30, 30, 30, 31, 31, 31, 32, 33, 33, 33, 34, 34, 34, 35, 36, 36, 36, 37, 37, 37, 38, 39, 39, 39, 40, 40, 40, 41, 42, 42, 42, 43, 43, 43, 44, 45, 45, 45, 46, 46, 46, 47, 48, 48, 48, 49, 49, 49, 50, 51, 51, 51, 52, 52, 52, 53, 54, 54, 54, 55, 55, 55, 56, 57, 57, 57, 58, 58, 58, 59, 60, 60, 60, 61, 61, 61, 62, 63, 63, 63, 64, 64, 64, 65, 66, 66, 66, 67, 67, 67, 68, 69, 69, 69, 70, 70, 70, 71, 72, 72, 72, 73, 73, 73, 74, 75, 75, 75, 76, 76, 76, 77, 78, 78, 78, 79, 79, 79, 80, 81, 81, 81, 82, 82, 82, 83, 84, 84, 84, 85, 85, 85, 86, 87, 87, 87, 88, 88, 88, 89, 90, 90, 90, 91, 91, 91, 92, 93, 93, 93, 94, 94, 94, 95, 96, 96, 96, 97, 97, 97, 98, 99, 99, 99, 100, 100, 100, 101, 102, 102, 102, 103, 103, 103, 104, 105, 105, 105, 106, 106, 106, 107]
c = [0, 1, 2, 0, 18, 19, 0, 1, 2, 0, 1, 20, 31, 1, 2, 0, 1, 2, 26, 32, 2, 3, 4, 5, 3, 20, 16, 3, 4, 5, 3, 4, 21, 34, 4, 5, 3, 4, 5, 13, 31, 5, 6, 7, 8, 6, 21, 22, 6, 7, 8, 6, 7, 23, 33, 7, 8, 6, 7, 8, 30, 34, 8, 9, 10, 11, 9, 23, 24, 9, 10, 11, 9, 10, 18, 32, 10, 11, 9, 10, 11, 28, 33, 11, 12, 13, 14, 12, 25, 26, 12, 13, 14, 12, 13, 31, 5, 13, 14, 12, 13, 14, 30, 29, 14, 15, 16, 17, 15, 27, 22, 15, 16, 17, 15, 16, 3, 20, 16, 17, 15, 16, 17, 19, 35, 17, 18, 19, 0, 18, 32, 10, 18, 19, 0, 18, 19, 35, 17, 19, 20, 16, 3, 20, 31, 1, 20, 21, 22, 6, 21, 34, 4, 21, 22, 6, 21, 22, 15, 27, 22, 23, 24, 9, 23, 33, 7, 23, 24, 9, 23, 24, 27, 35, 24, 25, 26, 12, 25, 29, 28, 25, 26, 12, 25, 26, 32, 2, 26, 27, 22, 15, 27, 35, 24, 27, 28, 25, 29, 28, 33, 11, 28, 29, 28, 25, 29, 14, 30, 29, 30, 29, 14, 30, 34, 8, 30, 31, 1, 20, 31, 5, 13, 31, 32, 10, 18, 32, 2, 26, 32, 33, 7, 23, 33, 11, 28, 33, 34, 4, 21, 34, 8, 30, 34, 35, 24, 27, 35, 17, 19, 35]
print(cvx.spmatrix(a, b, c))


[ 1.00e+00 -1.00e+00 -1.00e+00     0         0         0         0     ... ]
[ 1.00e+00     0         0         0         0         0         0     ... ]
[-1.00e+00     0         0         0         0         0         0     ... ]
[-1.00e+00  1.00e+00 -1.00e+00     0         0         0         0     ... ]
[    0      1.00e+00     0         0         0         0         0     ... ]
[    0     -1.00e+00     0         0         0         0         0     ... ]
[-1.00e+00 -1.00e+00  1.00e+00     0         0         0         0     ... ]
[    0         0      1.00e+00     0         0         0         0     ... ]
[    0         0     -1.00e+00     0         0         0         0     ... ]
[    0         0         0      1.00e+00 -1.00e+00 -1.00e+00     0     ... ]
[    0         0         0      1.00e+00     0         0         0     ... ]
[    0         0         0     -1.00e+00     0         0         0     ... ]
[    0         0         0     -1.00e+00  1.00e+00 -1.00e+00     0     ... ]